# EasyCV图像自监督训练-DINO

本文讲介绍如何利用EasyCV使用自监督算法[DINO](https://arxiv.org/abs/2104.14294)进行图像自监督模型的训练

## 运行环境要求

PAI-Pytorch镜像 or 原生Pytorch1.5+以上环境 GPU机器， 内存32G以上

## 安装依赖包

注: 在PAI-DSW docker中无需安装相关依赖，可跳过此部分 在本地notebook环境中执行


1. 首先，安装pytorch和对应版本的torchvision，支持Pytorch1.5.1以上版本

In [ ]:
# install pytorch and torch vision
! conda install --yes pytorch==1.10.0 torchvision==0.11.0 -c pytorch

2. 获取torch和cuda版本，安装对应版本的mmcv和nvidia-dali

In [ ]:
import torch
import os
os.environ['CUDA']='cu' + torch.version.cuda.replace('.', '')
os.environ['Torch']='torch'+torch.version.__version__.replace('+PAI', '')
!echo "cuda version: $CUDA"
!echo "pytorch version: $Torch"

In [ ]:
# install some python deps
! pip install mmcv-full==1.4.4 -f https://download.openmmlab.com/mmcv/dist/${CUDA}/${Torch}/index.html
! pip install http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/third_party/nvidia_dali_cuda100-0.25.0-1535750-py3-none-manylinux2014_x86_64.whl

3.  安装EasyCV算法包

In [ ]:
pip install pai-easycv

## 简单验证

In [ ]:
from easycv.apis import *

## EasyCV自监督训练

### 数据准备

自监督训练只需要提供无标注图片即可进行， 你可以下载[ImageNet](http://www.image-net.org/download-images) 数据，或者使用你自己的图片数据。需要提供一个包含若干图片的文件夹路径`p`，以及一个文件列表，文件列表中是每个图片相对图片目录`p`的路径

图片文件夹结构示例如下, 文件夹路径为`./images`

```shell
images/
├── 0001.jpg
├── 0002.jpg
├── 0003.jpg
|...
└── 9999.jpg
```

文件列表内容示例如下
```text
0001.jpg
0002.jpg
0003.jpg
...
9999.jpg
```

为了快速走通流程，我们也提供了一个小的示例数据集，执行如下命令下载解压

In [ ]:
! wget http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/imagenet_raw_demo/imagenet_raw_demo.tar.gz && tar -zxf imagenet_raw_demo.tar.gz

In [ ]:
# 重命名文件夹
! mv imagenet_raw_demo  imagenet_raw

### 模型训练

这个Demo中我们采用[mocov2](https://arxiv.org/abs/2003.04297)自监督算法训练ResNet50 主干网络， 下载示例配置文件

In [ ]:
! wget https://raw.githubusercontent.com/alibaba/EasyCV/master/configs/selfsup/dino/dino_deit_small_p16_8xb32_100e_jpg.py

为了缩短训练时间，打开配置文件 `mocov2_rn50_8xb32_200e_jpg.py`，修改`total_epoch`参数为20， 每隔1次迭代打印一次日志。

```python
# runtime settings
total_epochs = 20

# log config
log_config=dict(interval=1)
```

正式训练时，建议使用`单机8卡`配合该配置文件使用，如果要使用单机单卡，建议调小`optimizer.lr`初始学习率



In [ ]:
# 查看easycv安装位置
import easycv
print(easycv.__file__)

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=1 --master_port=29930 \
/home/pai/lib/python3.6/site-packages/easycv/tools/train.py dino_deit_small_p16_8xb32_100e_jpg.py --work_dir work_dir/selfsup/jpg/dino_deit_small_p16 --launcher pytorch

### 使用自监督模型进行特征抽取

#### 模型导出

模型导出会对自监督模型信息裁剪，保留特征抽取必要的backbone和head

In [ ]:
# 查看训练产生的模型文件
!ls work_dir/selfsup/jpg/dino_deit_small_p16/*.pth

In [ ]:
! python -m easycv.tools.export  dino_deit_small_p16_8xb32_100e_jpg.py work_dir/selfsup/jpg/dino_deit_small_p16/epoch_10.pth work_dir/selfsup/jpg/dino_deit_small_p16/epoch_10_export.pth

In [ ]:
#下载测试图片
! wget http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/pretrained_models/easycv/product_detection/248347732153_1040.jpg

In [ ]:
import cv2
from easycv.predictors.feature_extractor import TorchFeatureExtractor

# 修改output_ckpt指向
output_ckpt = 'work_dir/selfsup/jpg/dino_deit_small_p16/epoch_10_export.pth'
fe = TorchFeatureExtractor(output_ckpt)

img = cv2.imread('248347732153_1040.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
feature = fe.predict([img])
print(feature[0]['feature'].shape)
print(feature[0])

### 自监督预训练+ 图像分类finetune
参考EasyCV图像分类的demo， 在训练时加上--load_from 参数，使用自监督预训练的模型权重，  注意这里不需要使用

In [ ]:
! python -m easycv.tools.train  r50.py --work_dir work_dirs/classification/cifar10/dino_deit_small_p16  --load_from work_dir/selfsup/jpg/dino_deit_small_p16/epoch_10.pth